# Day 19 - Back to that CPU

We return to the CPU from [day 16](./Day%2016.ipynb); I remarked then (in my part 2 notes) that executing instructions was *certainly going to be straightforward, as there are no jumps (so no loops) in this instruction set.* Now we have jumps!

In [1]:
import operator
import re
from dataclasses import dataclass
from functools import partial
from typing import Callable, Iterable, Iterator, List, Mapping, Set, Sequence, Tuple, Type, Union

Instruction = Tuple[str, int, int, int]
OpcodeFunction = Union[Callable[[int, int], int], Callable[[int], int]]

@dataclass
class Opcode:
    # An opcode function takes one or two integers as input
    f: OpcodeFunction
    operands: str # 1 or 2 characters, i and r
    opcode: str = ''

    def __set_name__(self, owner: Type['CPU'], name: str) -> None:
        self.opcode = name[3:]
        owner.opcodes[self.opcode] = self

    def __repr__(self) -> str:
        return f'<opcode {self.opcode} {self.operands!r}>'
    
    def __get__(self, instance: 'CPU', owner: Type['CPU']) -> Callable[[int, int, int], None]:
        return partial(self.__call__, instance)

    def __call__(self, cpu: 'CPU', *ops: int) -> None:
        # operand C receives the output
        *ops, op_c = ops
        # map the oter operands to values
        value = {'r': cpu.registers.__getitem__, 'i': operator.pos}
        ops = (value[t](op) for t, op in zip(self.operands, ops))
        # execute the operand, store the integer result in the register
        # designated by C. Int is used to convert bool results to 0 or 1
        cpu.registers[op_c] = int(self.f(*ops))

def opcode(operands: str) -> Callable[[OpcodeFunction], Opcode]:
    """Operator operates on 1 or 2 operands
    
    Specify the operands as 'r' or 'i' for registers and immediate values.
    """
    def decorator(f: OpcodeFunction) -> Opcode:
        return Opcode(f, operands)
    return decorator

class CPU:
    registers: List[int]  # limited to 6
    opcodes: Mapping[str, Opcode] = {}

    def __init__(self) -> None:
        self.registers = [0, 0, 0, 0, 0, 0]
        self.bound: Mapping[str, Callable[[int, int, int], None]] = {
            name: opcode.__get__(self, None) for name, opcode in CPU.opcodes.items()
        }
    
    # Addition
    # addr (add register) stores into register C the result of adding register
    # A and register B.
    op_addr = opcode('rr')(operator.add)
    # addi (add immediate) stores into register C the result of adding
    # register A and value B.
    op_addi = opcode('ri')(operator.add)

    # Multiplication:
    # mulr (multiply register) stores into register C the result of
    # multiplying register A and register B.
    op_mulr = opcode('rr')(operator.mul)
    # muli (multiply immediate) stores into register C the result of
    # multiplying register A and value B.
    op_muli = opcode('ri')(operator.mul)

    # Bitwise AND:
    # banr (bitwise AND register) stores into register C the result of the
    # bitwise AND of register A and register B.
    op_banr = opcode('rr')(operator.and_)
    # bani (bitwise AND immediate) stores into register C the result of the
    # bitwise AND of register A and value B.
    op_bani = opcode('ri')(operator.and_)

    # Bitwise OR:
    # borr (bitwise OR register) stores into register C the result of the
    # bitwise OR of register A and register B.
    op_borr = opcode('rr')(operator.or_)
    # bori (bitwise OR immediate) stores into register C the result of the
    # bitwise OR of register A and value B.
    op_bori = opcode('ri')(operator.or_)

    # Assignment:
    # setr (set register) copies the contents of register A into register C.
    # (Input B is ignored.)
    op_setr = opcode('r')(operator.pos)  # pos is the identity function for int
    # seti (set immediate) stores value A into register C. (Input B is
    # ignored.)
    op_seti = opcode('i')(operator.pos)  # pos is the identity function for int

    # Greater-than testing:
    # gtir (greater-than immediate/register) sets register C to 1 if value A
    # is greater than register B. Otherwise, register C is set to 0.
    op_gtir = opcode('ir')(operator.gt)
    # gtri (greater-than register/immediate) sets register C to 1 if register
    # A is greater than value B. Otherwise, register C is set to 0.
    op_gtri = opcode('ri')(operator.gt)
    # gtrr (greater-than register/register) sets register C to 1 if register A
    # is greater than register B. Otherwise, register C is set to 0.
    op_gtrr = opcode('rr')(operator.gt)

    # Equality testing:
    # eqir (equal immediate/register) sets register C to 1 if value A is equal
    # to register B. Otherwise, register C is set to 0.
    op_eqir = opcode('ir')(operator.eq)
    # eqri (equal register/immediate) sets register C to 1 if register A is
    # equal to value B. Otherwise, register C is set to 0.
    op_eqri = opcode('ri')(operator.eq)
    # eqrr (equal register/register) sets register C to 1 if register A is
    # equal to register B. Otherwise, register C is set to 0.
    op_eqrr = opcode('rr')(operator.eq)

    def execute(self, ipregister: int, instructions: Sequence[Instruction], verbose: bool = False) -> int:
        opcodes = self.bound
        r = self.registers
        while 0 <= r[ipregister] < len(instructions):
            op, *operands = instructions[r[ipregister]]
            if verbose:
                print(
                    f"ip={r[ipregister]} {self.registers} {op} "
                    f"{operands[0]} {operands[1]} {operands[2]}", end=''
                )
            opcodes[op](*operands)
            if verbose:
                print(self.registers)
            r[ipregister] += 1
        return self.registers[0]

def parse_instructions(lines: Iterable[str]) -> Tuple[int, Sequence[Instruction]]:
    it = iter(lines)
    ipregister = int(next(it).rpartition(' ')[-1])
    instructions = []
    for line in it:
        opcode, *operands = line.split()
        instructions.append((opcode, *map(int, operands)))
    return ipregister, instructions

In [2]:
testprogram = parse_instructions('''\
#ip 0
seti 5 0 1
seti 6 0 2
addi 0 1 0
addr 1 2 3
setr 1 0 0
seti 8 0 4
seti 9 0 5'''.splitlines())
assert CPU().execute(*testprogram) == 7

In [3]:
import aocd

data = aocd.get_data(day=19, year=2018)
ipregister, instructions = parse_instructions(data.splitlines())

In [4]:
print('Part 1:', CPU().execute(ipregister, instructions))

Part 1: 912
